# Kapittel 7: Bygging av chatteapplikasjoner
## OpenAI API Hurtigstart

Denne notatboken er tilpasset fra [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) som inneholder notatbøker for tilgang til [Azure OpenAI](notebook-azure-openai.ipynb)-tjenester.

Python OpenAI API fungerer også med Azure OpenAI-modeller, med noen små endringer. Les mer om forskjellene her: [Hvordan bytte mellom OpenAI og Azure OpenAI-endepunkter med Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Oversikt  
"Store språkmodeller er funksjoner som kobler tekst til tekst. Gitt en inntekststreng, prøver en stor språkmodell å forutsi hvilken tekst som kommer etterpå"(1). Dette "kom-i-gang"-notatarket vil gi brukere en innføring i sentrale LLM-begreper, grunnleggende pakkekrav for å komme i gang med AML, en enkel introduksjon til promptdesign, og flere korte eksempler på ulike bruksområder.


## Innholdsfortegnelse  

[Oversikt](../../../../07-building-chat-applications/python)  
[Hvordan bruke OpenAI-tjenesten](../../../../07-building-chat-applications/python)  
[1. Opprette din OpenAI-tjeneste](../../../../07-building-chat-applications/python)  
[2. Installasjon](../../../../07-building-chat-applications/python)    
[3. Legitimasjon](../../../../07-building-chat-applications/python)  

[Bruksområder](../../../../07-building-chat-applications/python)    
[1. Oppsummere tekst](../../../../07-building-chat-applications/python)  
[2. Klassifisere tekst](../../../../07-building-chat-applications/python)  
[3. Generere nye produktnavn](../../../../07-building-chat-applications/python)  
[4. Finjustere en klassifiserer](../../../../07-building-chat-applications/python)  

[Referanser](../../../../07-building-chat-applications/python)


### Lag din første prompt  
Denne korte øvelsen gir en grunnleggende introduksjon til hvordan du sender inn forespørsler til en OpenAI-modell for en enkel oppgave, nemlig "oppsummering".


**Steg**:  
1. Installer OpenAI-biblioteket i ditt python-miljø  
2. Last inn vanlige hjelpebiblioteker og sett dine vanlige OpenAI-sikkerhetsnøkler for OpenAI-tjenesten du har opprettet  
3. Velg en modell for oppgaven din  
4. Lag en enkel prompt til modellen  
5. Send forespørselen din til modellens API!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Finne riktig modell  
GPT-3.5-turbo eller GPT-4-modellene kan forstå og generere naturlig språk.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Utforming av prompt  

"Magien med store språkmodeller er at ved å trene dem til å minimere denne prediksjonsfeilen over enorme mengder tekst, ender modellene opp med å lære konsepter som er nyttige for disse prediksjonene. For eksempel lærer de konsepter som"(1):

* hvordan man staver
* hvordan grammatikk fungerer
* hvordan man omformulerer
* hvordan man svarer på spørsmål
* hvordan man fører en samtale
* hvordan man skriver på mange språk
* hvordan man koder
* osv.

#### Hvordan styre en stor språkmodell  
"Av alle inputene til en stor språkmodell, er tekstprompten uten tvil den mest innflytelsesrike"(1).

Store språkmodeller kan promptes til å produsere utdata på flere måter:

Instruksjon: Fortell modellen hva du vil ha
Fullføring: Få modellen til å fullføre starten på det du ønsker
Demonstrasjon: Vis modellen hva du vil ha, enten med:
Noen få eksempler i prompten
Mange hundre eller tusen eksempler i et finjusteringsdatasett"



#### Det finnes tre grunnleggende retningslinjer for å lage gode prompt:

**Vis og fortell**. Gjør det tydelig hva du ønsker, enten gjennom instruksjoner, eksempler, eller en kombinasjon av begge. Hvis du vil at modellen skal rangere en liste med elementer i alfabetisk rekkefølge eller klassifisere et avsnitt etter stemning, vis at det er dette du vil.

**Gi gode data**. Hvis du prøver å lage en klassifiserer eller få modellen til å følge et mønster, sørg for at det er nok eksempler. Husk å korrekturlese eksemplene dine — modellen er som regel smart nok til å overse enkle stavefeil og gi deg et svar, men den kan også anta at dette er med vilje, og det kan påvirke svaret.

**Sjekk innstillingene dine.** Temperatur- og top_p-innstillingene styrer hvor deterministisk modellen er når den genererer et svar. Hvis du ber om et svar hvor det bare finnes ett riktig svar, bør du sette disse lavt. Hvis du ønsker mer varierte svar, kan du sette dem høyere. Den vanligste feilen folk gjør med disse innstillingene, er å tro at de styrer "smarthet" eller "kreativitet".


Kilde: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Oppsummer tekst  
#### Utfordring  
Oppsummer tekst ved å legge til en 'tl;dr:' på slutten av et tekstavsnitt. Legg merke til hvordan modellen forstår å utføre flere oppgaver uten ekstra instruksjoner. Du kan eksperimentere med mer beskrivende oppfordringer enn tl;dr for å endre modellens oppførsel og tilpasse oppsummeringen du får(3).  

Nylig arbeid har vist betydelige forbedringer på mange NLP-oppgaver og -benchmarker ved å forhåndstrene på et stort tekstkorpus, etterfulgt av finjustering på en spesifikk oppgave. Selv om denne metoden vanligvis er oppgaveuavhengig i arkitekturen, krever den fortsatt oppgavespesifikke finjusteringsdatasett med tusenvis eller titusenvis av eksempler. Til sammenligning kan mennesker vanligvis utføre en ny språklig oppgave med bare noen få eksempler eller enkle instruksjoner – noe dagens NLP-systemer fortsatt i stor grad sliter med. Her viser vi at å skalere opp språkmodeller gir betydelig bedre oppgaveuavhengig ytelse med få eksempler, og noen ganger til og med kan konkurrere med tidligere toppmoderne finjusteringsmetoder. 



Tl;dr


# Øvelser for flere brukstilfeller  
1. Oppsummer tekst  
2. Klassifiser tekst  
3. Lag nye produktnavn


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klassifiser tekst  
#### Utfordring  
Klassifiser elementer i kategorier som oppgis ved forespørselstidspunktet. I eksempelet under gir vi både kategoriene og teksten som skal klassifiseres i prompten (*playground_reference).

Kundehenvendelse: Hei, en av tastene på tastaturet mitt har gått i stykker nylig, og jeg trenger en erstatning:

Klassifisert kategori:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generer nye produktnavn
#### Utfordring
Lag produktnavn ut fra eksempler på ord. Her inkluderer vi informasjon om produktet vi skal lage navn til i prompten. Vi gir også et lignende eksempel for å vise mønsteret vi ønsker å få. Vi har også satt temperaturverdien høyt for å øke tilfeldigheten og få mer innovative svar.

Produktbeskrivelse: En milkshake-maskin for hjemmebruk
Stikkord: rask, sunn, kompakt.
Produktnavn: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Produktbeskrivelse: Et par sko som passer alle fotstørrelser.
Stikkord: tilpasningsdyktig, passform, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referanser  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Eksempler](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Beste praksis for finjustering av GPT-3 for tekstklassifisering](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# For mer hjelp  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Bidragsytere
* Louis Li



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på sitt originale språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell, menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
